In [58]:
import pandas as pd
import os

In [59]:
importFile = 'CombinedTrainingFrame_A2_A3_O3_egd.txt'
outName = 'CombinedTrainingFrame_A2_A3_O3_egd_short.txt'
df_import = pd.read_csv(importFile,names=['ID','label','extra'],sep='\t')

In [60]:
df_import['label'].value_counts()

label
17    250475
8     121850
0     112600
11     67150
3      66900
2      66400
9      53200
15     45550
14     29000
13     27100
7      19450
6      18625
12     12175
1       8500
5       4975
10      2225
16      1850
Name: count, dtype: int64

In [61]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s',1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES','NIFTI')+'.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath':NiftiPath, 'slicenum':slicenum})

In [62]:
df_NS = df_import.apply(extractNiftiFilepathAndSlicenum,axis=1, result_type='expand')

In [63]:
df = df_import.merge(df_NS,left_index=True,right_index=True,how='inner')

In [64]:
vc = df['label'].value_counts()
vc

label
17    250475
8     121850
0     112600
11     67150
3      66900
2      66400
9      53200
15     45550
14     29000
13     27100
7      19450
6      18625
12     12175
1       8500
5       4975
10      2225
16      1850
Name: count, dtype: int64

In [65]:
min = vc.iloc[-3]
min

4975

In [66]:
from sklearn.model_selection import  train_test_split

In [67]:
FinalFrame = pd.DataFrame(columns=['ID','label','extra'])
for l in df['label'].unique():
    temp = df[df['label']==l].groupby('NiftiPath')
    print(len(temp))
    try:
        keep, discard = train_test_split(list(range(len(temp))), train_size=int(min/25))
    except:
        keep = list(range(int(min/25)))
    for i,frame in enumerate(temp):
        if i in keep:
           FinalFrame =  pd.concat((FinalFrame,frame[1]),axis=0)

2668
4874
1822
4496
10019
1160
1078
74
199
745
2648
2686
2107
778
487
89
332


In [68]:
FinalFrame['label'].value_counts()

label
1     5100
13    5025
3     5000
0     4975
17    4975
14    4975
15    4975
8     4975
6     4975
2     4975
11    4975
9     4975
7     4975
12    4975
5     4975
10    2225
16    1850
Name: count, dtype: int64

In [69]:
FinalFrame = FinalFrame.reset_index(drop=True)

In [70]:
FinalFrame[['ID','label','extra']].to_csv(outName,header=False,index=False,sep='\t')

In [71]:
FinalFrame

,ID,label,extra,NiftiPath,slicenum
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,21.0
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,6.0
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,9.0
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,22.0
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,18.0
...,...,...,...,...,...
78895,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,4.0
78896,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,1.0
78897,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,6.0
78898,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,23.0
